# Calculate O2-O2_bar, T-T_bar, S-S_bar

In [1]:
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
tres = 'monthly'

### calculate (T-T_bar), (O2-O2_bar), (S-S_bar)

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_{tres}.nc'
ds_hind = xr.open_dataset(filename)
O2 = ds_hind['O2']
T = ds_hind['TEMP']
S = ds_hind['SALT']

In [4]:
nmon = ds_hind.dims['time']
aa = xr.DataArray(data=np.ones(nmon),coords=dict(time=('time',ds_hind.coords['time'].values)),dims='time')

In [5]:
filename = f'{path}/DPLE-FOSI_hindcast_O2_T_S_mean_dimension_reduced_1948-2017_{tres}.nc'
ds_m = xr.open_dataset(filename)
O2_mean = aa*ds_m['O2_mean']
T_mean = aa*ds_m['TEMP_mean']
S_mean = aa*ds_m['SALT_mean']
fddepth = aa*ds_m['3ddepth']

In [6]:
dO2 = O2 - O2_mean
dT = T - T_mean
dS = S - S_mean

In [7]:
filename = f'{path}/DPLE-FOSI_hindcast_AOU_and_AOU_mean_1948-2017_monthly.nc'
ds_hind = xr.open_dataset(filename)
AOU = ds_hind['AOU']
AOU_mean = ds_hind['AOU_mean']

In [8]:
dAOU = AOU - AOU_mean

In [9]:
dsf = xr.Dataset()
dsf['dO2'] = dO2
dsf['dT'] = dT
dsf['dS'] = dS
dsf['dAOU'] = dAOU
dsf['4ddepth'] = fddepth

In [10]:
dsf.load()

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, time: 840)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    dO2      (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dT       (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dS       (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dAOU     (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    4ddepth  (time, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [11]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_dO2_dT_dS_dAOU_1948-2017_{tres}.nc', mode='w')

## calculate partical derivatives

In [13]:
filename = f'{path}/DPLE-FOSI_hindcast_O2_T_S_mean_dimension_reduced_1948-2017_{tres}.nc'
ds_hind = xr.open_dataset(filename)
O2_mean = ds_hind['O2_mean']
T_mean = ds_hind['TEMP_mean']
S_mean = ds_hind['SALT_mean']
depth = ds_hind['3ddepth']
ds_hind.close()

In [14]:
Ac_dist = [10]
Eo_dist = np.arange(-.2,1.1,.1)
print(Ac_dist)
print(Eo_dist)

[10]
[-0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


In [15]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'dPhidO2_{nrow*i+j}'] =  Pd.dPhidO2(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022) #dEodT=0.022
    print('dPhidO2 done')
    dss[f'dPhidT_{nrow*i+j}'] = Pd.dPhidT(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022)  #dEodT=0.022
    print('dPhidT done')
    dss[f'dPhidS_{nrow*i+j}'] = Pd.dPhidS(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022)  #dEodT=0.022
    print('dPhidS done')

10 -0.2
dPhidO2 done
dPhidT done
dPhidS done
10 -0.1
dPhidO2 done
dPhidT done
dPhidS done
10 0.0
dPhidO2 done
dPhidT done
dPhidS done
10 0.10000000000000003
dPhidO2 done
dPhidT done
dPhidS done
10 0.2
dPhidO2 done
dPhidT done
dPhidS done
10 0.3
dPhidO2 done
dPhidT done
dPhidS done
10 0.4000000000000001
dPhidO2 done
dPhidT done
dPhidS done
10 0.5
dPhidO2 done
dPhidT done
dPhidS done
10 0.6000000000000001
dPhidO2 done
dPhidT done
dPhidS done
10 0.7
dPhidO2 done
dPhidT done
dPhidS done
10 0.8
dPhidO2 done
dPhidT done
dPhidS done
10 0.9000000000000001
dPhidO2 done
dPhidT done
dPhidS done
10 1.0000000000000002
dPhidO2 done
dPhidT done
dPhidS done
CPU times: user 53.6 s, sys: 136 ms, total: 53.7 s
Wall time: 56.5 s


In [16]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:     (z_t: 35, nlat: 140, nlon: 195)
Coordinates:
  * z_t         (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT        (nlat, nlon) float64 14.96 14.96 14.97 ... 70.43 70.42 70.42
    TLONG       (nlat, nlon) float64 101.2 102.3 103.4 ... 318.6 319.2 319.7
    ULAT        (nlat, nlon) float64 15.13 15.13 15.14 ... 70.64 70.63 70.63
    ULONG       (nlat, nlon) float64 101.7 102.9 104.0 ... 318.9 319.5 320.0
Dimensions without coordinates: nlat, nlon
Data variables: (12/39)
    dPhidO2_0   (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidT_0    (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidS_0    (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidO2_1   (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidT_1    (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidS_1    (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    ...          ...
    dPhidO2_11  (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidT_11   (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidS_11   (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidO2_12  (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidT_12   (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan
    dPhidS_12   (z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan

In [17]:
%%time
dss.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_dPhidO2_dPhidT_dPhidS_1948-2017_{tres}.nc', mode='w')

CPU times: user 57.2 ms, sys: 67.9 ms, total: 125 ms
Wall time: 538 ms
